In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
base_dir = "/content/drive/MyDrive/Data 298B Project Data/Test Dataset - Workbook 2"

In [ ]:
df = pd.read_csv(f"{base_dir}/image_metadata.csv")

In [ ]:
df.head()

,Id,Latitude,Longitude,Date and Time,Date
0,IMG_1240.JPG,22.503033,120.485581,2019:01:26 13:42:38,2019-01-26
1,IMG_1392.JPG,22.515956,120.487739,2019:01:26 15:07:09,2019-01-26
2,IMG_1326.JPG,22.501131,120.487236,2019:01:26 14:14:18,2019-01-26
3,IMG_20190308_114945.jpg,24.098481,120.669239,2019:03:08 11:49:45,2019-03-08
4,IMG_20190308_111908.jpg,24.099251,120.668691,2019:03:08 11:19:07,2019-03-08


In [ ]:
coordinates_and_dates = []

for _, row in df.iterrows():
    lat = round(row['Latitude'], 2)
    lon = round(row['Longitude'], 2)
    date_created = row['Date']
    coordinates_and_dates.append((lat, lon, date_created))

print(coordinates_and_dates)

[(22.5, 120.49, '2019-01-26'), (22.52, 120.49, '2019-01-26'), (22.5, 120.49, '2019-01-26'), (24.1, 120.67, '2019-03-08'), (24.1, 120.67, '2019-03-08'), (22.5, 120.49, '2019-01-26'), (22.5, 120.49, '2019-01-26'), (22.5, 120.49, '2019-01-26'), (24.1, 120.67, '2019-03-08'), (22.5, 120.49, '2019-01-26'), (22.5, 120.49, '2019-01-26'), (22.5, 120.49, '2019-01-26'), (22.52, 120.49, '2019-01-26'), (24.1, 120.67, '2019-03-08'), (22.52, 120.49, '2019-01-26'), (24.1, 120.67, '2019-03-08'), (22.5, 120.49, '2019-01-26'), (24.07, 120.66, '2018-12-27'), (22.5, 120.49, '2019-01-26'), (24.07, 120.65, '2018-12-27')]


In [ ]:
# Getting rid of duplicate tuples

from collections import OrderedDict

# Using an OrderedDict to remove duplicates while maintaining order. Keys are the tuples.
unique_coordinates_and_dates_ordered = list(OrderedDict.fromkeys(coordinates_and_dates))

print(unique_coordinates_and_dates_ordered)

[(22.5, 120.49, '2019-01-26'), (22.52, 120.49, '2019-01-26'), (24.1, 120.67, '2019-03-08'), (24.07, 120.66, '2018-12-27'), (24.07, 120.65, '2018-12-27')]


In [ ]:
print(len(unique_coordinates_and_dates_ordered))

5


In [ ]:
import requests
import csv
from datetime import datetime, timedelta
import time
start_time = time.time()

# Visual Crossing Weather API Key
API_KEY = ""
API_ENDPOINT = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'

file_path = f"{base_dir}/weather_data.csv"

# Sample list of tuples containing latitude, longitude, and date
locations_dates = [
    (24.07, 120.66, '2018-12-27'),
    (24.08, 120.65, '2018-12-27')
]

# Preparing the CSV file at the specified file path
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Latitude", "Longitude", "Date", "Avg Temp 14d", "Avg Humidity 14d", "Total Precipitation 14d", "Avg Wind Speed 14d"])

    for latitude, longitude, date_str in unique_coordinates_and_dates_ordered:
        # Calculating the date range to obtain the data
        end_date = datetime.strptime(date_str, '%Y-%m-%d')
        start_date = end_date - timedelta(days=14)

        # Formatting the dates for the API request
        start_date_str = start_date.strftime('%Y-%m-%d')
        end_date_str = (end_date - timedelta(days=1)).strftime('%Y-%m-%d')  # Excluding the given date itself

        # Making the API request
        response = requests.get(f"{API_ENDPOINT}{latitude},{longitude}/{start_date_str}/{end_date_str}?unitGroup=metric&include=days&key={API_KEY}&contentType=json")
        weather_data = response.json()

        # Initializing variables for the calculations
        total_temp = 0
        total_humidity = 0
        total_precipitation = 0
        total_wind_speed = 0
        days_counted = 0

        # Accumulating the weather data
        for day in weather_data['days']:
            total_temp += day['temp']
            total_humidity += day['humidity']
            total_precipitation += day.get('precip', 0)
            total_wind_speed += day['windspeed']
            days_counted += 1

        # Calculating the averages
        avg_temp = total_temp / days_counted if days_counted else 0
        avg_humidity = total_humidity / days_counted if days_counted else 0
        avg_wind_speed = total_wind_speed / days_counted if days_counted else 0

        writer.writerow([latitude, longitude, date_str, avg_temp, avg_humidity, total_precipitation, avg_wind_speed])

print(f"CSV file has been created at {file_path} with the weather data.")

elapsed_time = time.time() - start_time
print(f"It took {elapsed_time:.4f} seconds to obtain the weather data from the image metadata.")

CSV file has been created at /content/drive/MyDrive/Data 298B Project Data/Test Dataset - Workbook 2/weather_data.csv with the weather data.
It took 1.3073 seconds to obtain the weather data from the image metadata.
